In [16]:
import pandas as pd
import geopandas as gpd
import requests
#Endpoint for API with key attached
apifile = open('../data/apikey.txt')
apikey = apifile.read()
url = 'https://api.openei.org/utility_rates?version=3&format=json&' + apikey

In [17]:
utilities = pd.read_csv('../data/utilities.csv',index_col=0)
names = iter(range(0,18))
for num in names:
    file = pd.read_csv('../data/rates' + str(num) + '.csv', na_values='No', index_col=0, dtype={'GEOID10':'str','eiaid':'str'})
    utilities = utilities.append(file)
#utilities.drop_duplicates(inplace=True)

In [18]:
utilities = utilities.drop(columns='region_name')

In [19]:
utilities['GEOID10'].value_counts()

94591.0    24
1819.0     24
45505.0    24
88504.0    24
76339.0    24
           ..
83073       1
49042       1
61057       1
11620       1
16171       1
Name: GEOID10, Length: 6806, dtype: int64

In [20]:
utilities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22766 entries, 0 to 4
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GEOID10  21584 non-null  object
 1   utility  21190 non-null  object
 2   eiaid    21190 non-null  object
dtypes: object(3)
memory usage: 711.4+ KB


In [21]:
metros = gpd.read_file('../data/metro/tl_2020_us_uac10.shp')

In [22]:
metros['GEOID10'].nunique()

3601

In [25]:
missing = metros[~metros['GEOID10'].isin(utilities['GEOID10'])]
#missing = metros[metros['GEOID10'].isin(utilities[utilities['utility']=='No']['GEOID10'].astype('int').astype('str'))]
missing = metros[metros['GEOID10']=='40429']

In [26]:
len(missing)

1

In [27]:
utility_region = pd.DataFrame()
x = []
for index, row in missing.iterrows():
    params = {'sector':'Residential',          
         'lat':row.INTPTLAT10,
         'lon':row.INTPTLON10
         }
    response = requests.get(url, params = params)
    print(str(index)+' - ' +str(response))
    if len(response.json()['items']):
        df = pd.json_normalize(response.json()['items'])
        df['GEOID10'] = row['GEOID10']
        utility_region = utility_region.append(df.loc[:,['GEOID10','utility','eiaid']].drop_duplicates())
    else:
        print('else')
        utility_region = utility_region.append({'GEOID10': row['GEOID10'],'utility':'No','eiaid':'No'}, ignore_index=True)


936 - <Response [200]>
else


In [ ]:
utility_region['eiaid'].value_counts()

In [ ]:
utility_region

In [ ]:
utilities = utilities.append(utility_region)

In [13]:
utilities

,GEOID10,utility,eiaid
0,24310.0,Commonwealth Edison Co,4110
1,27847.0,"City of Escanaba, Michigan (Utility Company)",24558
2,18100.0,"City of Clintonville, Wisconsin (Utility Company)",3814
3,6166.0,Duke Energy Indiana Inc,15470
4,75270.0,Pacific Gas & Electric Co,14328
...,...,...,...
68,95482,Consumers Energy Co,4254
20,56980,Consumers Energy Co,4254
20,10675,Consumers Energy Co,4254
27,31450,Jersey Central Power & Lt Co (New Jersey),9726


In [15]:

utilities.dropna().astype({'GEOID10':'int'}).astype({'GEOID10':'str'})
utilities

,GEOID10,utility,eiaid
0,24310.0,Commonwealth Edison Co,4110
1,27847.0,"City of Escanaba, Michigan (Utility Company)",24558
2,18100.0,"City of Clintonville, Wisconsin (Utility Company)",3814
3,6166.0,Duke Energy Indiana Inc,15470
4,75270.0,Pacific Gas & Electric Co,14328
...,...,...,...
68,95482,Consumers Energy Co,4254
20,56980,Consumers Energy Co,4254
20,10675,Consumers Energy Co,4254
27,31450,Jersey Central Power & Lt Co (New Jersey),9726


In [ ]:
utilities.to_csv('../data/utilities.csv')

In [ ]:
utilities

In [ ]:
utilities[utilities['utility']=='No']['GEOID10'].astype('int').astype('str')

In [28]:
response.json()

{'items': [],
 'errors': ['Utility company lookup failed for lat/lon +29.7843079/-095.3935305']}